# Create a dataset of the latest news

## imports

In [ ]:
! pip install newsapi-python

In [ ]:
! pip install yfinance

In [ ]:
from newsapi import NewsApiClient
import newsapi
#from newsapi.newsapi_client import NewsApiClient
from datetime import date, timedelta, datetime
import pandas as pd

# Obtain access key to newAPI
* get a free key on the website https://newsapi.org
* NEWS_API_KEY = personal api key

In [ ]:
# Show full output in Colab
# https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab
pd.set_option('display.max_colwidth',1000)

In [ ]:
# Init news api
NEWS_API_KEY = '1900869fa01647fca0bdc19b4550daa0'

# '1900869fa01647fca0bdc19b4550daa0'
# '71234eb576cd465da04f8050a01be06a'

### See all sources available

In [ ]:
news_client = NewsApiClient(api_key= NEWS_API_KEY)
# the list of available categories to choose from
print('categories:', newsapi.const.categories)
# the list of available sources in the business domain
# sources = news_client.get_sources(category='business', language='en', country='us')
# source_names = [source['id'] for source in sources['sources']]

# dividing the list into large sources that require the entire api call and small sources that can be used together
large_sources = ['bloomberg', 'business-insider']
small_sources = ['financial-post', 'fortune', 'the-wall-street-journal']
source_names = large_sources + small_sources
print(source_names)

# Download all articles from the past month

### You can download all articles sorted by date, or all articles sorted by company from the snp500

In [ ]:
# from tqdm import tqdm
# source_names = large_sources + [",".join(small_sources)]

# #https://newsapi.org/docs/endpoints/everything
# tot_articles = []
# # datetime.strptime('10-Apr-2021','%d-%b-%Y')
# for i in tqdm(range(31)):
#     end_date = date.today() - timedelta(days=i)
#     start_date = date.today() - timedelta(days=i+1)
#     for source in source_names:
#         articles = news_client.get_everything(from_param=start_date.isoformat(), to=end_date.isoformat(), language="en", sources=source, sort_by="relevancy")
#         tot_articles.extend(articles['articles'])

In [ ]:
from tqdm import tqdm
tot_articles = []
# datetime.strptime('10-Apr-2021','%d-%b-%Y')
end_date = date.today()
start_date = date.today() - timedelta(days=31)
articles = news_client.get_everything(from_param=start_date.isoformat(), to=end_date.isoformat(), language="en", sources=",".join(source_names), sort_by="relevancy")
tot_articles.extend(articles['articles'])

In [ ]:
len(tot_articles)

In [ ]:
articles_df = pd.DataFrame(tot_articles)

In [ ]:
# apply a map function to the source column to get the source name
articles_df['source'] = articles_df['source'].map(lambda x: x['id'])

In [ ]:
# count the number of rows in a dataframe
print(articles_df.shape)
# get all unique values in a column
print(articles_df['source'].unique())
articles_df.head()

## Scrape websites to get the actual articles

In [ ]:
print(tot_articles[0]['url'])

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint

# At this point we will have some search results from the API. Take the first search result...
first_result = tot_articles[0]

# ...and download the HTML for it, again with requests
r2 = requests.get(first_result["url"])

# Check if the request was successful
if r2.status_code == 200:
    # We now have the article HTML, so parse it with BeautifulSoup
    soup = BeautifulSoup(r2.content, "html.parser")
    # Done! The article content can be extracted by searching the HTML
    article_content = soup.find("div", {"id": "article-body"}).text
    print(article_content)
else:
    # Handle unsuccessful request
    print("Error:", r2.status_code)


In [ ]:
print(soup)

Save results csv for later work

In [ ]:
# save dataframe to csv without index
articles_df.to_csv("30_days_ago"+end_date.isoformat()+".csv", index=False)

articles_df.to_csv("30_days_ago"+end_date.isoformat()+".csv")

In [ ]:
! zip {"30_days_ago"+end_date.isoformat()+".zip"} {"30_days_ago"+end_date.isoformat()+".csv"}